### API MINI-Project

In this project, we focus on equities data from the Frankfurt Stock Exchange (FSE), available from Quandl API. More specifically, we address some questions related to the stock prices of a company called [Carl Zeiss Meditec](https://www.zeiss.com/meditec/int/home.html), which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery. The company is listed under the stock ticker AFX_X.

To send requests to Quandl API and get access to the data, we first register on [Quandl website](http://www.quandl.com) and obtain an API key that we store as a string:

In [1]:
# Store the API key as a string 
API_KEY = '...'

#### Quick Glimpse into the Data

Before we address the tasks of this mini-project, we pull out a sample of 2 days from the data in order to explore the structure of the data returned.

In [2]:
#First, import the relevant modules
import requests

# Now, call the Quandl API and pull out a small sample of two days to get a glimpse into the returned JSON structure 
url='https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?limit=2&api_key='+API_KEY
r=requests.get(url)
json_data=r.json()

In [3]:
# Inspect the JSON structure of the object you created
print(json_data)

{'dataset_data': {'limit': 2, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2019-10-07', 'frequency': 'daily', 'data': [['2019-10-07', 108.5, 110.3, 107.5, 108.8, None, 97154.0, 10561664.0, None, None, None], ['2019-10-04', 105.0, 109.5, 104.4, 108.2, None, 198283.0, 21355196.0, None, None, None]], 'collapse': None, 'order': None}}


We see that we can find the column names as follows:

In [4]:
print('The column names are as follows:\n')
print(json_data['dataset_data']['column_names'])

The column names are as follows:

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


On the other hand, the data can be retrieved as follows:

In [5]:
print('The data is given as a list of lists:\n')
print(json_data['dataset_data']['data'])

The data is given as a list of lists:

[['2019-10-07', 108.5, 110.3, 107.5, 108.8, None, 97154.0, 10561664.0, None, None, None], ['2019-10-04', 105.0, 109.5, 104.4, 108.2, None, 198283.0, 21355196.0, None, None, None]]


#### Project Tasks

We now consider the following tasks:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

**Task 1**: We first collect the data for the whole year 2017.

In [6]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017
url='https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY
r=requests.get(url)
json_data=r.json()

**Task 2**: Using defaultdict from the collections library, we convert the JSON object into a Python dictionary.

In [7]:
# 2. Convert the returned JSON object into a Python dictionary

# import defaultdict
from collections import defaultdict

# create empty dictionary
data=defaultdict(list)

# find the column names
colnames=json_data['dataset_data']['column_names']

# retrieve the data stored as list of lists
datalists=json_data['dataset_data']['data']

# loop over each list and store it in the dictionary 'data' under the right key
for datalist in datalists:
    for key,value in zip(colnames,datalist):
        data[key].append(value)

**Task 3**: We now find the highest and lowest opening prices for the stock for the year of 2017. The opening prices are paired with the key 'Open' in the data dictionary. Since some of the opening prices are stored as None, we first remove them and then find the maximum and minimum values. 

In [8]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period

# remove the None values and then find the maximum and minimum values of opening prices
OpenPrices=list(filter(None,data['Open']))
highest=max(OpenPrices)
lowest=min(OpenPrices)

print('The highest opening price was:',highest)
print('The lowest opening price was:',lowest)

The highest opening price was: 53.11
The lowest opening price was: 34.0


**Task 4**: We here find the largest change in any one day (based on High and Low price). The highest price in any given day is given under the key 'High' and the lowest price per day is given under the key 'Low'. To find the largest change in any given day, we subtract the lowest prices from the highest prices on each day and then find the largest difference.

In [9]:
# 4. What was the largest change in any one day (based on High and Low price)?

# subtract the entries of data['Low'] from the entries of data['High'] and then find the maximum change
max_change=max([h-l for h,l in zip(data['High'],data['Low'])])

print('The largest change in any one day was', max_change)

The largest change in any one day was 2.8100000000000023


**Task 5**: We here find the largest change between any two days based on Closing Price. To get the closing prices of each day, we use the data under the key 'Close'. We then subtract the values of each two consecutive days and then find the maximum absolute change. 

In [10]:
# 5. What was the largest change between any two days (based on Closing Price)?

# data['Close'] corresponds to closing prices from '2017-12-29' to '2017-01-02'

# data['Close'][:-1]: has the closing prices from '2017-12-29' to '2017-01-03'
# data['Close'][1:]: has the closing prices from '2017-12-28' to '2017-01-02'
# by subtracting the above two slices, we obtain the change in closing prices between any two days

max_change_2=max([abs(day2-day1) for day2,day1 in zip(data['Close'][:-1],data['Close'][1:])])

print('The largest change between any two days was', max_change_2)

The largest change between any two days was 2.559999999999995


**Task 6**: We here compute the average daily trading volume during this year.

In [11]:
# 6. What was the average daily trading volume during this year?

avg_volume=sum(data['Traded Volume'])/len(data['Traded Volume'])

print('The average daily trading volume was',avg_volume)

The average daily trading volume was 89124.33725490196


**Task 7**: We here find the median trading volume. We first sort the trading volumes and since the length of the data is odd (255), the median is the middle value in the sorted volume data. 

In [12]:
# 7. What was the median trading volume during this year?

# sort the traded volume
volume_sorted=sorted(data['Traded Volume'])

# find the index of the middle value (the length of the data is odd)
mid_index=len(volume_sorted)//2

# median is the middle value
median=volume_sorted[mid_index]

print('The median trading volume was ',median )

The median trading volume was  76286.0
